In [ ]:
import numpy as np
import numba as nb
import matplotlib.pyplot as plt
from numba import jit
from numba import njit
from numba import prange
import scipy.special as sp
import scipy.integrate as spi
from scipy.optimize import brentq, fsolve
from scipy.integrate import quad
import time
import os

import tensorflow as tf
import tensornetwork as tn

# Set TensorNetwork to use the TensorFlow backend (needed for autodiff)
tn.set_default_backend("tensorflow")
from tensornetwork import FiniteMPS

In [22]:
N = 5 #number of qubits

# each site tensor has shape (chi_left=1, d=2, chi_right=1)
# Initialise all the qubits in |0>, i.e., A[0]=1, A[1]=0
prod_tensors = []
for _ in range(N):
    # shape (2,) → reshape into (1,2,1)
    v = tf.constant([1.0, 0.0], dtype=tf.complex64)
    prod_tensors.append(tf.reshape(v, (1,2,1)))

# initialize FiniteMPS (in canonical form)
psi = FiniteMPS(prod_tensors, canonicalize=True)

# Hadamard
H_gate = tf.constant([[1.0,  1.0],
                 [1.0, -1.0]], dtype=tf.complex64) / tf.cast(tf.sqrt(2.0), tf.complex64)

# apply it on the first site by contracting over the physical index:
# new A[0]_{α,s',β} = Σ_s H[s',s] · A[0]_{α,s,β}
A0 = psi.tensors[0] # shape (1,2,1)
A0 = tf.tensordot(H_gate, A0, axes=([1],[1]))  # → (2,1,1)
psi.tensors[0] = tf.transpose(A0, (1,0,2))  

print("Applied Hadamard on site 1; MPS bonds =", 
      [t.shape for t in psi.tensors])


#Test single-site obsrvables
qubit_indx = 1 #choice of the qubit
psi.position(qubit_indx)
A = psi.tensors[qubit_indx]
rho = tf.tensordot(tf.math.conj(A), A, axes=([0,2],[0,2]))

X = tf.constant([[0.,1.],[1.,0.]], tf.complex64)
Y = tf.constant([
        [0 + 0j, -1 + 0j],
        [1 + 0j,  0 + 0j]
    ], dtype=tf.complex64)
Z = tf.constant([[1.,0.],[0.,-1.]], tf.complex64)

avg_X = tf.linalg.trace(tf.matmul(rho, X))
avg_Y = tf.linalg.trace(tf.matmul(rho, Y))
avg_Z = tf.linalg.trace(tf.matmul(rho, Z))

x = tf.math.real(avg_X).numpy()    # gives a NumPy float
y = tf.math.real(avg_Y).numpy()
z = tf.math.real(avg_Z).numpy()
print(f"<X> = {x:.6f},  <Y> = {y:.6f},  <Z> = {z:.6f}")


Applied Hadamard on site 1; MPS bonds = [TensorShape([1, 2, 1]), TensorShape([1, 2, 1]), TensorShape([1, 2, 1]), TensorShape([1, 2, 1]), TensorShape([1, 2, 1])]
<X> = 0.000000,  <Y> = 0.000000,  <Z> = 1.000000
